In [4]:
#Importing Libs and Creating Session
from pyspark.sql import SparkSession
from pyspark.sql.functions import substring,to_date,concat,lit,col,split

spark = SparkSession.builder.appName("DataTransformation").getOrCreate()

In [5]:
#Reading CSV file
df = spark.read.format("csv").load("C:\\Users\\Ahmed\\OneDrive\\Desktop\\SingleTable\\cleanDS",header=True)

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/C:/Users/Ahmed/OneDrive/Desktop/SingleTable/cleanDS.

In [ ]:
#Printing Schema
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Type: string (nullable = true)



In [ ]:
#Printing first 20 rows
df.show(20)

+----------+-----+--------------------+--------------------+--------------------+
|      Date| Time|            Location|               Route|                Type|
+----------+-----+--------------------+--------------------+--------------------+
|01/19/1930|18:23|Oceanside, Califo...|Aqua Caliente, Me...|Ford 5-AT-C Tri M...|
|03/31/1931|10:45|      Bazaar, Kansas|Kansas City - Wic...|Fokker F10A Trimotor|
|08/31/1934|23:42|  Amazonia, Missouri|  Omaha - St. Joseph|   Stinson  SM-6000B|
|05/06/1935|03:30|   Atlanta, Missouri|Los Angeles - Alb...|    Douglas DC-2-112|
|08/14/1935|23:45|  Near Gilmer, Texas|   Dallas -  Atlanta|     Stinson Model A|
|10/07/1935|02:19|Near Cheyenne, Wy...|Salt Lake City - ...|         Boeing 247D|
|01/14/1936|19:32|   Goodwin, Arkansas|Newark, NJ - Fort...|    Douglas DC-2-120|
|04/07/1936|10:20|Uniontown, Pennsy...|Newark - Los Angeles|    Douglas DC-2-112|
|12/15/1936|03:14|Near Salt Lake Ci...|Burbank - Las Veg...|         Boeing 247D|
|12/19/1936|20:4

In [ ]:
#Transforming the Date column from DD/MM/YYYY format to YYYY format for easier operations
df = df.withColumn("Date", substring(df["Date"], 7, 4))


In [ ]:
df.show()

+----+-----+--------------------+--------------------+--------------------+
|Date| Time|            Location|               Route|                Type|
+----+-----+--------------------+--------------------+--------------------+
|1930|18:23|Oceanside, Califo...|Aqua Caliente, Me...|Ford 5-AT-C Tri M...|
|1931|10:45|      Bazaar, Kansas|Kansas City - Wic...|Fokker F10A Trimotor|
|1934|23:42|  Amazonia, Missouri|  Omaha - St. Joseph|   Stinson  SM-6000B|
|1935|03:30|   Atlanta, Missouri|Los Angeles - Alb...|    Douglas DC-2-112|
|1935|23:45|  Near Gilmer, Texas|   Dallas -  Atlanta|     Stinson Model A|
|1935|02:19|Near Cheyenne, Wy...|Salt Lake City - ...|         Boeing 247D|
|1936|19:32|   Goodwin, Arkansas|Newark, NJ - Fort...|    Douglas DC-2-120|
|1936|10:20|Uniontown, Pennsy...|Newark - Los Angeles|    Douglas DC-2-112|
|1936|03:14|Near Salt Lake Ci...|Burbank - Las Veg...|         Boeing 247D|
|1936|20:47|Near Milford, Pen...|Miami, FL - Charl...|    Douglas DC-2-112|
|1936|19:38|

In [ ]:


# Created a new Dataframe Grouping by the year and number of crashes in each year
CrashesInYear = df.groupBy("Date").count()
CrashesInYear = CrashesInYear.withColumnRenamed("count", "CrashesInYear")
# Join the original DataFrame with the crash_counts DataFrame on the Date column
df = df.join(CrashesInYear, "Date", "left_outer")
# Show the resulting DataFrame
df.show()

+----+-----+--------------------+--------------------+--------------------+-------------+
|Date| Time|            Location|               Route|                Type|CrashesInYear|
+----+-----+--------------------+--------------------+--------------------+-------------+
|1930|18:23|Oceanside, Califo...|Aqua Caliente, Me...|Ford 5-AT-C Tri M...|            1|
|1931|10:45|      Bazaar, Kansas|Kansas City - Wic...|Fokker F10A Trimotor|            1|
|1934|23:42|  Amazonia, Missouri|  Omaha - St. Joseph|   Stinson  SM-6000B|            1|
|1935|03:30|   Atlanta, Missouri|Los Angeles - Alb...|    Douglas DC-2-112|            3|
|1935|23:45|  Near Gilmer, Texas|   Dallas -  Atlanta|     Stinson Model A|            3|
|1935|02:19|Near Cheyenne, Wy...|Salt Lake City - ...|         Boeing 247D|            3|
|1936|19:32|   Goodwin, Arkansas|Newark, NJ - Fort...|    Douglas DC-2-120|            5|
|1936|10:20|Uniontown, Pennsy...|Newark - Los Angeles|    Douglas DC-2-112|            5|
|1936|03:1

In [ ]:
#Split the Location Column and Only Kept the State's Name
df = df.withColumn("Location", split("Location", ",").getItem(1))
#Split the Route Column and Only Kept the State's Name
df = df.withColumn("Route", split("Route", "-").getItem(1))
df.show(10)

+----+-----+-------------+----------------+--------------------+-------------+
|Date| Time|     Location|           Route|                Type|CrashesInYear|
+----+-----+-------------+----------------+--------------------+-------------+
|1930|18:23|   California|     Los Angeles|Ford 5-AT-C Tri M...|            1|
|1931|10:45|       Kansas|        Wichita |Fokker F10A Trimotor|            1|
|1934|23:42|     Missouri|      St. Joseph|   Stinson  SM-6000B|            1|
|1935|03:30|     Missouri|    Albuquerque |    Douglas DC-2-112|            3|
|1935|23:45|        Texas|         Atlanta|     Stinson Model A|            3|
|1935|02:19|      Wyoming|        Cheyenne|         Boeing 247D|            3|
|1936|19:32|     Arkansas|  Fort Worth, TX|    Douglas DC-2-120|            5|
|1936|10:20| Pennsylvania|     Los Angeles|    Douglas DC-2-112|            5|
|1936|03:14|         Utah|      Las Vegas |         Boeing 247D|            5|
|1936|20:47| Pennsylvania| Charleston, SC |    Dougl

In [ ]:
# Created a new Dataframe Grouping by the year and number of crashes in each year
CrashesInPlace = df.groupBy("Location").count()
CrashesInPlace = CrashesInPlace.withColumnRenamed("count", "CrashesInPlace")
# Join the original DataFrame with the crash_counts DataFrame on the Date column
df = df.join(CrashesInPlace, "Location", "left_outer")
# Show the resulting DataFrame
df.show()

+-------------+----+-----+----------------+--------------------+-------------+--------------+
|     Location|Date| Time|           Route|                Type|CrashesInYear|CrashesInPlace|
+-------------+----+-----+----------------+--------------------+-------------+--------------+
|   California|1930|18:23|     Los Angeles|Ford 5-AT-C Tri M...|            1|            43|
|       Kansas|1931|10:45|        Wichita |Fokker F10A Trimotor|            1|             1|
|     Missouri|1934|23:42|      St. Joseph|   Stinson  SM-6000B|            1|             9|
|     Missouri|1935|03:30|    Albuquerque |    Douglas DC-2-112|            3|             9|
|        Texas|1935|23:45|         Atlanta|     Stinson Model A|            3|            13|
|      Wyoming|1935|02:19|        Cheyenne|         Boeing 247D|            3|             8|
|     Arkansas|1936|19:32|  Fort Worth, TX|    Douglas DC-2-120|            5|             3|
| Pennsylvania|1936|10:20|     Los Angeles|    Douglas DC-2-

In [ ]:
# Created a new Dataframe Grouping by the year and number of crashes in each year
FailureRate = df.groupBy("Type").count()
FailureRate = FailureRate.withColumnRenamed("count", "FailureRate")
# Join the original DataFrame with the crash_counts DataFrame on the Date column
df = df.join(FailureRate, "Type", "left_outer")
# Show the resulting DataFrame
df.show(10)

+--------------------+--------------------+----+-------+--------------------+-------------+--------------+-----------+
|                Type|            Location|Date|   Time|               Route|CrashesInYear|CrashesInPlace|FailureRate|
+--------------------+--------------------+----+-------+--------------------+-------------+--------------+-----------+
|Ford 5-AT-C Tri M...|          California|1930|  18:23|         Los Angeles|            1|            43|          1|
|Fokker F10A Trimotor|              Kansas|1931|  10:45|            Wichita |            1|             1|          1|
|   Stinson  SM-6000B|            Missouri|1934|  23:42|          St. Joseph|            1|             9|          1|
|    Douglas DC-2-112|            Missouri|1935|  03:30|        Albuquerque |            3|             9|          6|
|     Stinson Model A|               Texas|1935|  23:45|             Atlanta|            3|            13|          1|
|         Boeing 247D|             Wyoming|1935|

In [ ]:
#Cleaning the Data and Dropping Unnecessary Columns
finalDF = df.dropna().drop("Route","Time").sort("Date",ascending=True)
#Displaying the Transformed Dataset
finalDF.select("Date","CrashesInYear","Location","CrashesInPlace","Type","FailureRate").show(915)

+----+-------------+--------------------+--------------+--------------------+-----------+
|Date|CrashesInYear|            Location|CrashesInPlace|                Type|FailureRate|
+----+-------------+--------------------+--------------+--------------------+-----------+
|1930|            1|          California|            43|Ford 5-AT-C Tri M...|          1|
|1931|            1|              Kansas|             1|Fokker F10A Trimotor|          1|
|1934|            1|            Missouri|             9|   Stinson  SM-6000B|          1|
|1935|            3|            Missouri|             9|    Douglas DC-2-112|          6|
|1935|            3|               Texas|            13|     Stinson Model A|          1|
|1935|            3|             Wyoming|             8|         Boeing 247D|          2|
|1936|            5|            Arkansas|             3|    Douglas DC-2-120|          1|
|1936|            5|        Pennsylvania|            14|    Douglas DC-2-112|          6|
|1936|    

In [ ]:
#Splitting the new dataset to 2 separate datasets
percent = [0.5,0.5]
splitCSV = finalDF.randomSplit(weights=percent,seed=21)

#Assigning the split CSV files to Variables
DS1 = splitCSV[0]
DS2 = splitCSV[1]

#Saving new datasets
DS1.write.csv("C:\\Users\\Ahmed\\OneDrive\\Desktop\\SingleTable\\DS1", header=True, mode="overwrite")
DS2.write.csv("C:\\Users\\Ahmed\\OneDrive\\Desktop\\SingleTable\\DS2", header=True, mode="overwrite")

In [ ]:
#Stopping spark session
spark.stop()